In [ ]:
import zipfile, os, cv2
import numpy as np
import tensorflow as tf
tf.executing_eagerly()
from tensorflow import keras
# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from keras import backend as K
K.set_image_data_format('channels_last')

In [ ]:
size = 192
depth = 32
ALL_data_paths = [
    os.path.join(os.getcwd(), "is0001-is0154_mask_and_image/", x)
    for x in sorted(os.listdir("is0001-is0154_mask_and_image/"))
]
print("MRI scans dataset: " + str(len(ALL_data_paths)//2) + ' ALL')
mask = []
image = []
for i in ALL_data_paths:
    if 's.nii.gz' in i:
        mask.append(i)
    else:
        image.append(i)
print(len(mask), len(image))

In [ ]:
image_path = image

In [ ]:
import nibabel as nib
from skimage import morphology
from scipy import ndimage
from PIL import Image

def normalize(volume, norm_type):
    """Normalize the volume"""
#     min = np.min(volume)
#     max = np.max(volume)
#     volume[volume < min] = min
#     volume[volume > max] = max
#     volume = (volume - min) / (max - min)
    if norm_type == 'zero_mean':
        img_o = np.float32(volume.copy())
        m = np.mean(img_o)
        s = np.std(img_o)
        volume = np.divide((img_o - m), s)
    elif norm_type == 'div_by_max':
        volume = np.divide(volume, np.percentile(volume,98))
    volume = volume.astype("float32")
    return volume

def remove_noise_from_image(file_path):
    image = nib.load(file_path)
    if len(image.shape) == 4:
        image = image.get_fdata()
        width,height,queue,_ = image.shape
        image = image[:,:,:,1]
        image = np.reshape(image,(width,height,queue))
    else:
        image = image.get_fdata()
        pass
    shape = image.shape
    for i in range(shape[2]):
        image_2d = image[:, :, i]
#         mask = image_2d <=20
        mask = image_2d<=10
        selem = morphology.disk(2)

        segmentation = morphology.dilation(mask, selem)
        labels, label_nb = ndimage.label(segmentation)

        mask = labels ==0
        mask = morphology.dilation(mask, selem)
        mask = ndimage.morphology.binary_fill_holes(mask)
        mask = morphology.dilation(mask, selem)

        image[:, :, i] = mask * image_2d
    image = normalize(image,"div_by_max")
    return image

def resize_volume(img,size,depth):
    """Resize across z-axis"""
    # Set the desired depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
        # Rotate img shape = (height, wight, depth)
    for i in range(img.shape[2]):
        img[:,:,i] = np.fliplr(np.flipud(img[:,:,i]))
#     img = ndimage.rotate(img, 180, reshape=False, mode="nearest")
    img = ndimage.zoom(img, (size/current_height, size/current_width, 1), order=0)
    return img

In [ ]:
def process_scan(path):
#     # Resize width, height and depth
    volume = remove_noise_from_image(path)
    volume = resize_volume(volume,size,depth)
#   add only black background mri image
    if volume.shape[2]!=depth:
        add_black_num = depth - volume.shape[2]
        volume = np.transpose(volume)
        for i in range(add_black_num):
            add_black_ = np.expand_dims(np.zeros((volume.shape[2],volume.shape[2])),axis=0)
            volume = np.concatenate((volume, add_black_), axis = 0)
        volume = np.transpose(volume)
    volume = np.transpose(volume)
    print(path)
    print(f"rebuild shape: {volume.shape}")
    return volume
def mask_scan(path):
#     print(path)
    image = nib.load(path)
    
    if len(image.shape) == 4:
        image = image.get_fdata()
        width,height,queue,_ = image.shape
        image = image[:,:,:,1]
        image = np.reshape(image,(width,height,queue))
    else:
        image = image.get_fdata()
        pass
    image = resize_volume(image,size,depth)
    shape = image.shape
#   add only black background mri image
    if image.shape[2]!=depth:
        add_black_num = depth - image.shape[2]
        image = np.transpose(image)
        for i in range(add_black_num):
            add_black_ = np.expand_dims(np.zeros((image.shape[2],image.shape[2])),axis=0)
            image = np.concatenate((image, add_black_), axis = 0)
        image = np.transpose(image)
    image = np.transpose(image)
    print(path)
    print(f"rebuild shape: {image.shape}")
    return image

In [ ]:
# image_arr = np.array([process_scan(path) for path in image])
# print("Done")
# mask_arr = np.array([mask_scan(path) for path in mask])
# print("Done")

In [ ]:
# np.save('np_multi/image_arr_154', image_arr)
# np.save('np_multi/mask_arr_154', mask_arr)
image_arr = np.load('np_multi/image_arr_154.npy').astype(np.float32)
mask_arr = np.load('np_multi/mask_arr_154.npy').astype(np.float32)
print(image_arr.shape, mask_arr.shape)

In [ ]:
print(mask_arr.shape)
# mask_arr = np.reshape(mask_arr,(86,32,192,192,1))
plt.figure(figsize=(20,20))
image_arr_ = image_arr[20]
print(f"pixel max = {np.max(image_arr_)} pixel min = {np.min(image_arr_)}")
for i in range(image_arr_.shape[0]-24):
    plt.subplot(1,image_arr_.shape[0]-24,i+1)
    plt.axis('off')
    plt.imshow(image_arr_[i],cmap='gray')


In [ ]:
plt.figure(figsize=(20,20))
mask_arr_ = mask_arr[20]
print(f"pixel max = {np.max(mask_arr_)} pixel min = {np.min(mask_arr_)}")
for i in range(mask_arr_.shape[0]-24):
    plt.subplot(1,mask_arr_.shape[0]-24,i+1)
    plt.axis('off')
    plt.imshow(mask_arr_[i],cmap='gray')

In [ ]:
from keras.utils.np_utils import to_categorical
image_arr = np.transpose(image_arr,(0,2,3,1))
mask_arr = np.transpose(mask_arr,(0,2,3,1))

image_arr = np.expand_dims(image_arr, axis=-1)
mask_arr = np.expand_dims(mask_arr, axis=-1)
print(image_arr.shape, mask_arr.shape)

In [ ]:
from keras.utils.np_utils import to_categorical
# seed = np.random.randint(200)
seed = 987
# random shuffle dataset
# seed: cv1=1 cv2=123 cv3=456 cv4=789 cv5=987
np.random.seed(seed)
np.random.shuffle(image_arr) #image
np.random.seed(seed)
np.random.shuffle(mask_arr) #label
np.random.seed(seed)
np.random.shuffle(image_path)
print(image_path[0])
# mask_arr = to_categorical(mask_arr, 2 ,dtype='uint8')

In [ ]:
# np.save('np_multi/image_arr_cv2', image_arr)
# np.save('np_multi/mask__arr_cv2', mask_arr)
# np.save('np_multi/image_path_cv2', image_path)

In [ ]:
# np.save('np_multi/train_path_cv2_aug', image_path_train)

In [ ]:
image_path_train = image_path[:105]
image_path_valid = image_path[105:]
x_train = image_arr[:105]
y_train = mask_arr[:105]
x_val = image_arr[105:]
y_val = mask_arr[105:]

In [ ]:
# t1 cv1
# /ssd1/cnn/Classification/3d_mask_classification/is0001-is0154_mask_and_image/is0088o.nii.gz
# t1 cv2
# /ssd1/cnn/Classification/3d_mask_classification/is0001-is0154_mask_and_image/is0005o.nii.gz
# t1 cv3
# /ssd1/cnn/Classification/3d_mask_classification/is0001-is0154_mask_and_image/is0047o.nii.gz

In [ ]:
# cv1
# /ssd1/cnn/Classification/3d_mask_classification/is0001-is0100_mask_and_image/is0047o.nii.gz
# cv2
# /ssd1/cnn/Classification/3d_mask_classification/is0001-is0100_mask_and_image/is0076o.nii.gz

In [ ]:
imgs_train = x_train  # scale masks to [0, 1]
imgs_valid = x_val# scale masks to [0, 1]
imgs_mask_train = y_train # scale masks to [0, 1]
imgs_mask_valid = y_val  # scale masks to [0, 1]
print(len(image_path_train))
print(imgs_train.shape)
print(imgs_mask_train.shape)
print(imgs_valid.shape)
print(imgs_mask_valid.shape)

In [ ]:
# from ipywidgets import interact
# %matplotlib inline
# def browse_image(images, labels):
# #     shape = depth, height, wight
#     n = images.shape[2]
#     def view_image(i):
        
#         fig, ax = plt.subplots(nrows = 1, ncols = 2)
#         ax[0].imshow(images[...,i], cmap = 'gray', interpolation = 'nearest')
#         ax[0].set_title('X Slice: %s' %i)
#         ax[1].imshow(labels[...,i], cmap = 'gray', interpolation = 'nearest')
#         ax[1].set_title('Y Slice: %s' %i)
#     interact(view_image, i = (0,n-1))
# for i in range(imgs_train.ndim):
#     browse_image(imgs_train[i,...,0], imgs_mask_train[i,...,0])

In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# for i in range(105):
#     print(image_path_train[i])
#     for j in range(32): 
#         if np.max(imgs_train[i][:,:,j])>0:
            
#             plt.figure(figsize=(12,12))
#             plt.subplot(1,3,1)
#             plt.imshow(np.squeeze(imgs_train[i][:,:,j]),cmap='gray')
#             plt.title('Original Image')
#             plt.subplot(1,3,2)
#             plt.imshow(np.squeeze(imgs_mask_train[i][:,:,j]),cmap='gray')
#             plt.title('Original Mask')
#             plt.show()

In [ ]:
from augmented import generator

image_aug = generator.customImageDataGenerator(
            rotation_range = 20,
#             brightness_range=[0.5,1.0]
            )

mask_aug = generator.customImageDataGenerator(
#             featurewise_center=True,
#             featurewise_std_normalization=True,
            rotation_range = 20,
#             brightness_range=[0.5,1.5]
            )

image_aug_valid = generator.customImageDataGenerator(
            )

mask_aug_valid = generator.customImageDataGenerator(
            )

In [ ]:
bs = 1

X_train_datagen = image_aug.flow(imgs_train, batch_size=bs, seed=seed)
Y_train_datagen = mask_aug.flow(imgs_mask_train, batch_size=bs, seed=seed)
train_generator = zip(X_train_datagen, Y_train_datagen)

X_valid_datagen = image_aug_valid.flow(imgs_valid, batch_size=bs, seed=seed)
Y_valid_datagen = mask_aug_valid.flow(imgs_mask_valid, batch_size=bs, seed=seed)
valid_generator = zip(X_valid_datagen, Y_valid_datagen)

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_p_bce(in_gt, in_pred):
    return 1e-3*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)

# def dice_coef_loss(y_true, y_pred):
#     return 1 - dice_coef(y_true, y_pred)


# def tversky(y_true, y_pred):
#     smooth=1
#     alpha=0.7
#     y_true_pos = K.flatten(y_true)
#     y_pred_pos = K.flatten(y_pred)
#     true_pos = K.sum(y_true_pos * y_pred_pos)
#     false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
#     false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
#     return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

# def tversky_loss(y_true, y_pred):
#     return 1 - tversky(y_true, y_pred)

In [ ]:
from model_library.model_3d_denseunet import threed_unet
from keras.optimizers import Adam
from keras import backend as K
from keras.metrics import binary_crossentropy
# fitting shape [[slice, w, h, c], class]
model = threed_unet()
learning_rate = 1e-5
epoch = 150
learning_decay_rate = learning_rate/epoch
model.compile(optimizer=Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, 
                             decay=learning_decay_rate, amsgrad=False), 
           loss=dice_p_bce, metrics=['accuracy',dice_coef])

# model.compile(optimizer=Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, 
#                              decay=learning_decay_rate, amsgrad=False), 
#            loss=general_dice_loss, metrics=['accuracy',general_dice])

# model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard
weight_dir = 'checkpoint'
checkpoint_name = 'Unet_mri-best_cv5_aug_g-dl_dense_3d_154_t1'
# model.load_weights(os.path.join(weight_dir,checkpoint_name+'.hdf5'))
if not os.path.exists(weight_dir):
    os.mkdir(weight_dir)
# checkpoint_name = 'Unet_mri-epoch:{epoch:02d}-loss:{loss:.2f}-Dice:{dice_coef:.4f}.hdf5'

model_checkpoint = ModelCheckpoint(os.path.join(weight_dir+'/metric_try3',f"{checkpoint_name}.hdf5"), 
                                   monitor='val_loss', mode="auto", verbose=0, save_best_only=True)
logdir = os.path.join("checkpoint/tensorboard2/", checkpoint_name)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1,
                                         embeddings_freq=0,embeddings_layer_names=None,)


In [ ]:
print('-'*30,'\nFitting model...\n','-'*30)
# history = model.fit(imgs_train, imgs_mask_train, batch_size=1, epochs=epoch, verbose=1, 
#                     shuffle=True, validation_data=(imgs_valid,imgs_mask_valid), 
#                     callbacks=[model_checkpoint,tensorboard_callback])

history = model.fit(train_generator, epochs=epoch, 
                    verbose=0,
                    steps_per_epoch= (len(imgs_train))//bs,
                    shuffle=True, validation_data=valid_generator,
                    validation_steps= len(imgs_valid)//bs,          
                    callbacks=[model_checkpoint,tensorboard_callback])


# history = model.fit(train_dataset, epochs=epoch, verbose=2, 
#                     shuffle=True, validation_data=validation_dataset, 
#                     callbacks=[model_checkpoint,tensorboard_callback])


In [ ]:
print('-'*30)
print('Loading and preprocessing test data...')
print('-'*30)


imgs_test = imgs_valid
imgs_mask_test = imgs_mask_valid


print('-'*30)
print('Loading saved weights...')
print('-'*30)

model = threed_unet()
weight_dir = 'checkpoint/metric_try3'
checkpoint_name = checkpoint_name + '.hdf5'
model.load_weights(os.path.join(weight_dir,checkpoint_name ))

In [ ]:
Results = model.predict(imgs_test, batch_size=1, verbose=1)

In [ ]:
print(Results.shape)

In [ ]:
import matplotlib.pyplot as plt
for j in range(Results.shape[0]):
    count = 1
    for i in range(32):
        if np.max(imgs_test[j][:,:,i])>0:
            
            plt.figure(figsize=(12,12))
            plt.subplot(1,3,1)
            plt.imshow(np.squeeze(imgs_test[j][:,:,i]), cmap='gray')
            plt.title('Original Image')
            plt.subplot(1,3,2)
            plt.imshow(np.squeeze(imgs_mask_test[j][:,:,i]), cmap='gray')
            plt.title('Original Mask')
            plt.subplot(1,3,3)
            plt.imshow(np.squeeze(Results[j][:,:,i]) > .5, cmap='gray')

            plt.title('Prediction')
            plt.show()
            print(image_path_valid[j][-14:]+"_"+str(count)+"↑")
            count+=1